# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 13 2022 8:52AM,248527,12,HH-37491,Hush Hush,Released
1,Oct 13 2022 8:52AM,248527,12,HH-37492,Hush Hush,Released
2,Oct 13 2022 8:52AM,248527,12,HH-37493,Hush Hush,Released
3,Oct 13 2022 8:52AM,248527,12,HH-37496,Hush Hush,Released
4,Oct 13 2022 8:52AM,248527,12,HH-37497,Hush Hush,Released
5,Oct 13 2022 8:52AM,248527,12,HH-37498,Hush Hush,Released
6,Oct 13 2022 8:52AM,248527,12,HH-37499,Hush Hush,Released
7,Oct 13 2022 8:52AM,248527,12,HH-37500,Hush Hush,Released
8,Oct 13 2022 8:52AM,248527,12,HH-37501,Hush Hush,Released
9,Oct 13 2022 8:52AM,248527,12,HH-37502,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,248522,Executing,2
31,248522,Released,4
32,248525,Released,1
33,248526,Released,2
34,248527,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248521,NaN,NaN,1.0
248522,NaN,2.0,4.0
248525,NaN,NaN,1.0
248526,NaN,NaN,2.0
248527,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248081,0.0,1.0,0.0
248083,0.0,0.0,1.0
248087,0.0,0.0,1.0
248089,0.0,0.0,1.0
248091,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248081,0,1,0
248083,0,0,1
248087,0,0,1
248089,0,0,1
248091,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248081,0,1,0
1,248083,0,0,1
2,248087,0,0,1
3,248089,0,0,1
4,248091,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248081,,1,
1,248083,,,1
2,248087,,,1
3,248089,,,1
4,248091,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 13 2022 8:52AM,248527,12,Hush Hush
11,Oct 13 2022 8:42AM,248526,19,ACG North America LLC
13,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation
46,Oct 13 2022 8:30AM,248515,10,ISDIN Corporation
56,Oct 13 2022 8:23AM,248525,19,ACG North America LLC
57,Oct 13 2022 7:46AM,248522,19,"AdvaGen Pharma, Ltd"
63,Oct 12 2022 4:56PM,248521,21,"NBTY Global, Inc."
64,Oct 12 2022 4:43PM,248091,21,"NBTY Global, Inc."
65,Oct 12 2022 4:42PM,248089,21,"NBTY Global, Inc."
66,Oct 12 2022 4:37PM,248087,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 13 2022 8:52AM,248527,12,Hush Hush,,,11
1,Oct 13 2022 8:42AM,248526,19,ACG North America LLC,,,2
2,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation,,,33
3,Oct 13 2022 8:30AM,248515,10,ISDIN Corporation,,,10
4,Oct 13 2022 8:23AM,248525,19,ACG North America LLC,,,1
5,Oct 13 2022 7:46AM,248522,19,"AdvaGen Pharma, Ltd",,2,4
6,Oct 12 2022 4:56PM,248521,21,"NBTY Global, Inc.",,,1
7,Oct 12 2022 4:43PM,248091,21,"NBTY Global, Inc.",,,1
8,Oct 12 2022 4:42PM,248089,21,"NBTY Global, Inc.",,,1
9,Oct 12 2022 4:37PM,248087,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 8:52AM,248527,12,Hush Hush,11,,
1,Oct 13 2022 8:42AM,248526,19,ACG North America LLC,2,,
2,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation,33,,
3,Oct 13 2022 8:30AM,248515,10,ISDIN Corporation,10,,
4,Oct 13 2022 8:23AM,248525,19,ACG North America LLC,1,,
5,Oct 13 2022 7:46AM,248522,19,"AdvaGen Pharma, Ltd",4,2,
6,Oct 12 2022 4:56PM,248521,21,"NBTY Global, Inc.",1,,
7,Oct 12 2022 4:43PM,248091,21,"NBTY Global, Inc.",1,,
8,Oct 12 2022 4:42PM,248089,21,"NBTY Global, Inc.",1,,
9,Oct 12 2022 4:37PM,248087,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 8:52AM,248527,12,Hush Hush,11,,
1,Oct 13 2022 8:42AM,248526,19,ACG North America LLC,2,,
2,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation,33,,
3,Oct 13 2022 8:30AM,248515,10,ISDIN Corporation,10,,
4,Oct 13 2022 8:23AM,248525,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 8:52AM,248527,12,Hush Hush,11.0,NaN,NaN
1,Oct 13 2022 8:42AM,248526,19,ACG North America LLC,2.0,NaN,NaN
2,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation,33.0,NaN,NaN
3,Oct 13 2022 8:30AM,248515,10,ISDIN Corporation,10.0,NaN,NaN
4,Oct 13 2022 8:23AM,248525,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 8:52AM,248527,12,Hush Hush,11.0,0.0,0.0
1,Oct 13 2022 8:42AM,248526,19,ACG North America LLC,2.0,0.0,0.0
2,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation,33.0,0.0,0.0
3,Oct 13 2022 8:30AM,248515,10,ISDIN Corporation,10.0,0.0,0.0
4,Oct 13 2022 8:23AM,248525,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2236285,72.0,4.0,31.0
12,248527,11.0,0.0,0.0
15,248265,2.0,0.0,28.0
16,496924,2.0,0.0,0.0
18,248462,1.0,0.0,0.0
19,1739515,15.0,6.0,0.0
21,1488952,5.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2236285,72.0,4.0,31.0
1,12,248527,11.0,0.0,0.0
2,15,248265,2.0,0.0,28.0
3,16,496924,2.0,0.0,0.0
4,18,248462,1.0,0.0,0.0
5,19,1739515,15.0,6.0,0.0
6,21,1488952,5.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,72.0,4.0,31.0
1,12,11.0,0.0,0.0
2,15,2.0,0.0,28.0
3,16,2.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,15.0,6.0,0.0
6,21,5.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,72.0
1,12,Released,11.0
2,15,Released,2.0
3,16,Released,2.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,21
Status,,,,,,,
Completed,31.0,0.0,28.0,0.0,0.0,0.0,0.0
Executing,4.0,0.0,0.0,0.0,0.0,6.0,1.0
Released,72.0,11.0,2.0,2.0,1.0,15.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,21
0,Completed,31.0,0.0,28.0,0.0,0.0,0.0,0.0
1,Executing,4.0,0.0,0.0,0.0,0.0,6.0,1.0
2,Released,72.0,11.0,2.0,2.0,1.0,15.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,21
0,Completed,31.0,0.0,28.0,0.0,0.0,0.0,0.0
1,Executing,4.0,0.0,0.0,0.0,0.0,6.0,1.0
2,Released,72.0,11.0,2.0,2.0,1.0,15.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()